# Notebook 08: Audio - Text-to-Speech (TTS)

**Learning Objectives:**
- Generate speech from text using TTS models
- Use SpeechT5 for natural-sounding speech
- Control voice characteristics with speaker embeddings
- Save and play generated audio

## Prerequisites

### Hardware Requirements

| Model Option | Model Name | Size | Min RAM | Recommended Setup | Notes |
|--------------|------------|------|---------|-------------------|-------|
| **CPU/GPU** | microsoft/speecht5_tts | 200MB | 4GB | 6GB VRAM (RTX 4080) | Good quality |

### Software Requirements
- Python 3.8+
- Libraries: `transformers`, `torch`, `soundfile`, `datasets`

In [ ]:
import torch
from transformers import SpeechT5Processor, SpeechT5ForTextToSpeech, SpeechT5HifiGan, set_seed
from datasets import load_dataset
import soundfile as sf
import warnings
warnings.filterwarnings('ignore')

# Set seed for reproducibility
set_seed(1103)

print(f"PyTorch version: {torch.__version__}")
print(f"CUDA available: {torch.cuda.is_available()}")

## Expected Behaviors

### First Time Running
- **Model Download**: ~200MB for SpeechT5 TTS (~2-3 minutes)
- Also downloads vocoder (~50MB)
- Downloads speaker embeddings dataset (~100MB)
- Total: ~350MB first run

### Setup Cell Output
```
PyTorch version: 2.x.x
CUDA available: True/False
```

### Model Loading
```
Model loaded on: cpu (or cuda)
Speaker embeddings loaded
```
- **CPU**: 10-15 seconds (loads 3 components)
- **GPU**: 5-8 seconds

### Audio Output
- Saves as WAV file at 16kHz sample rate
- File size: ~1.5MB per minute of audio
- Can be played in Jupyter with `IPython.display.Audio()`

### Speech Quality
- **Natural sounding** for most English text
- **Intonation**: Reasonably natural, slight robotic quality
- **Pronunciation**: 90-95% accurate for common words
- **Prosody**: Handles punctuation (pauses, emphasis)

### Voice Characteristics
- Controlled by speaker embeddings
- Different embeddings = different voice characteristics
- Current notebooks use single voice (can experiment with others)
- Voice quality is consistent across generations

### Performance
- **Short sentence** (10-20 words):
  - CPU: 3-5 seconds
  - GPU: 1-2 seconds
- **Long paragraph** (100+ words):
  - CPU: 15-25 seconds
  - GPU: 5-8 seconds

### Text Limitations
- **Works best with**: Clear, grammatical English
- **Maximum length**: ~200 words per generation (split longer text)
- **Numbers**: Speaks digits individually (e.g., "1234" → "one two three four")
- **Abbreviations**: May mispronounce (expand them first)

### Common Issues
- **Monotone speech**: Try different speaker embeddings
- **Mispronunciations**: Phonetic spelling may help
- **Unnatural pauses**: Check punctuation
- **Cutoff**: Text too long, split into chunks

### Expected Output Files
- `output.wav`, `hello.wav`, etc.
- Saved in notebook directory
- Can open with any audio player

### Quality Factors
- Punctuation affects pacing and intonation
- ALL CAPS may sound overly emphasized
- Question marks add rising intonation

In [ ]:
MODEL_NAME = "microsoft/speecht5_tts"
VOCODER_NAME = "microsoft/speecht5_hifigan"

print(f"Selected model: {MODEL_NAME}")

In [ ]:
# Load model components
print("Loading TTS model...")
processor = SpeechT5Processor.from_pretrained(MODEL_NAME)
model = SpeechT5ForTextToSpeech.from_pretrained(MODEL_NAME)
vocoder = SpeechT5HifiGan.from_pretrained(VOCODER_NAME)

device = "cuda" if torch.cuda.is_available() else "cpu"
model = model.to(device)
vocoder = vocoder.to(device)

print(f"Model loaded on: {device}")

In [ ]:
# Load speaker embeddings
embeddings_dataset = load_dataset("Matthijs/cmu-arctic-xvectors", split="validation")
speaker_embeddings = torch.tensor(embeddings_dataset[7306]["xvector"]).unsqueeze(0).to(device)

print("Speaker embeddings loaded")

In [ ]:
def text_to_speech(text, output_file="output.wav"):
    """
    Convert text to speech and save to file.
    """
    # Process text
    inputs = processor(text=text, return_tensors="pt").to(device)
    
    # Generate speech
    with torch.no_grad():
        speech = model.generate_speech(
            inputs["input_ids"], 
            speaker_embeddings, 
            vocoder=vocoder
        )
    
    # Save audio
    speech_np = speech.cpu().numpy()
    sf.write(output_file, speech_np, samplerate=16000)
    
    print(f"Audio saved to: {output_file}")
    return speech_np

In [ ]:
# Example: Generate speech
text = "Hello, welcome to the HuggingFace tutorial on text-to-speech!"

audio = text_to_speech(text, "hello.wav")
print(f"\nGenerated {len(audio)} audio samples")

In [ ]:
# Generate multiple phrases
phrases = [
    "Artificial intelligence is transforming the world.",
    "Machine learning models can now understand and generate speech.",
    "This technology has many practical applications."
]

print("=== GENERATING SPEECH FOR MULTIPLE PHRASES ===")
for i, phrase in enumerate(phrases, 1):
    output_file = f"phrase_{i}.wav"
    text_to_speech(phrase, output_file)
    print(f"  {i}. {phrase[:50]}...")

In [ ]:
# Try different speaker embeddings
def try_different_voices(text, num_voices=3):
    """
    Generate speech with different voice characteristics.
    """
    print(f"\n=== GENERATING WITH DIFFERENT VOICES ===")
    
    for i in range(min(num_voices, len(embeddings_dataset))):
        speaker_emb = torch.tensor(embeddings_dataset[i]["xvector"]).unsqueeze(0).to(device)
        
        inputs = processor(text=text, return_tensors="pt").to(device)
        
        with torch.no_grad():
            speech = model.generate_speech(
                inputs["input_ids"],
                speaker_emb,
                vocoder=vocoder
            )
        
        output_file = f"voice_{i+1}.wav"
        sf.write(output_file, speech.cpu().numpy(), samplerate=16000)
        print(f"  Voice {i+1} saved to: {output_file}")

# Test
try_different_voices("This is a test of different voice characteristics.", num_voices=3)

## State-of-the-Art Open Models (Not Covered)

While SpeechT5 is a solid foundation for TTS, there are several cutting-edge models that offer superior naturalness, multilingual support, voice cloning capabilities, and even emotional expression. These represent the frontier of text-to-speech technology.

### Top SOTA Text-to-Speech Models

#### 1. 🎤 XTTS v2 (Coqui AI)
**Multilingual TTS with voice cloning**
- **Why it's special**: Zero-shot voice cloning from 3-6 seconds of audio, 16 languages
- **Performance**: Near-human naturalness, real-time capable, excellent prosody
- **Model Card**: [coqui/XTTS-v2](https://huggingface.co/coqui/XTTS-v2)
- **Paper**: [XTTS: A Massively Multilingual Zero-Shot Text-to-Speech Model](https://arxiv.org/abs/2406.04904)
- **Size**: 1.8GB

#### 2. 🎵 Bark (Suno AI)
**GPT-style TTS with music, sound effects, and emotions**
- **Why it's special**: Generates laughter, music, sound effects, multiple languages, emotional speech
- **Performance**: Highly expressive, can include non-speech sounds, ~10s for short sentences
- **Model Card**: [suno/bark](https://huggingface.co/suno/bark)
- **GitHub**: [suno-ai/bark](https://github.com/suno-ai/bark)
- **Size**: 2.8GB (full model)

#### 3. 🔊 VITS (Variational Inference TTS)
**End-to-end neural TTS with high quality**
- **Why it's special**: Single-stage model (no vocoder needed), fast inference, excellent quality
- **Performance**: Real-time factor < 0.1 (10x faster than real-time), natural prosody
- **Model Card**: Multiple variants on HuggingFace (language-specific)
- **Paper**: [Conditional Variational Autoencoder with Adversarial Learning](https://arxiv.org/abs/2106.06103)
- **Size**: ~400MB

#### 4. 🌍 YourTTS
**Zero-shot multi-speaker and multilingual TTS**
- **Why it's special**: Zero-shot voice cloning, 1107 speakers, multilingual support
- **Performance**: High speaker similarity, good cross-lingual capabilities
- **Model Card**: [coqui/YourTTS](https://huggingface.co/models?search=yourtts)
- **Paper**: [YourTTS: Towards Zero-Shot Multi-Speaker TTS](https://arxiv.org/abs/2112.02418)
- **Size**: ~500MB

### Why Not Covered?

These advanced models require:
- **GPU Memory**: 8-16GB VRAM for real-time inference
- **Generation Time**: 10-60 seconds per sentence on CPU
- **Disk Space**: 1.8-2.8GB per model
- **Complex Setup**: Voice cloning requires reference audio preprocessing
- **Specialized Use Cases**: Features like emotion/music may not be needed for basic TTS

SpeechT5 provides excellent quality for standard TTS use cases!

### Learning Path Recommendation

1. **Start here**: Master SpeechT5 (this notebook)
2. **Voice cloning**: Try XTTS v2 if you have GPU and need custom voices
3. **Expressiveness**: Experiment with Bark for emotional/creative applications
4. **Speed focus**: Use VITS for fast, high-quality production TTS
5. **Research**: Explore YourTTS for multilingual voice transfer

### Benchmarks & Quality Metrics

- **Mean Opinion Score (MOS)** - Naturalness (1-5 scale):
  - SpeechT5: 4.0-4.2
  - XTTS v2: 4.3-4.5
  - Bark: 4.2-4.4 (with expressiveness)
  - VITS: 4.1-4.3
  - YourTTS: 4.0-4.2

- **Real-Time Factor (RTF)** - Lower is faster:
  - SpeechT5: ~0.3 (GPU), ~2.0 (CPU)
  - XTTS v2: ~0.2 (GPU), ~8.0 (CPU)
  - VITS: ~0.1 (GPU), ~1.5 (CPU)
  - Bark: ~1.0 (GPU), ~15.0 (CPU)

- **Speaker Similarity** (voice cloning):
  - XTTS v2: 85-90%
  - YourTTS: 80-85%
  - Bark: N/A (not designed for cloning specific voices)

- **Explore benchmarks**: [Papers With Code - Speech Synthesis](https://paperswithcode.com/task/speech-synthesis)

### Quick Comparison Table

| Model | Size | Speed | Quality | Languages | Special Features |
|-------|------|-------|---------|-----------|------------------|
| **SpeechT5** ⭐ | 200MB | Fast | Good | English | Learning-friendly |
| **VITS** | 400MB | Very Fast | Great | Multi* | Production-ready |
| **YourTTS** | 500MB | Fast | Good | 16+ | Zero-shot cloning |
| **XTTS v2** | 1.8GB | Medium | Excellent | 16 | Voice cloning, multilingual |
| **Bark** | 2.8GB | Slow | Great | Multi | Music, emotions, SFX |

*Separate models per language

### Language Support

| Model | English | Multi-language | Voice Cloning | Emotion |
|-------|---------|----------------|---------------|---------|
| **SpeechT5** | ✅ | ❌ | ❌ | ❌ |
| **VITS** | ✅ | ✅* | ❌ | ❌ |
| **YourTTS** | ✅ | ✅ | ✅ | ❌ |
| **XTTS v2** | ✅ | ✅ (16 langs) | ✅ | ⚠️ (limited) |
| **Bark** | ✅ | ✅ | ❌ | ✅ |

### Special Capabilities

**🌟 What makes these models unique:**

- **XTTS v2**: Clone anyone's voice from 6 seconds of audio
- **Bark**: Add `[laughs]`, `[music]`, `[sigh]` tags for rich audio
- **VITS**: Fastest inference, ideal for interactive applications
- **YourTTS**: Cross-lingual voice transfer (speak any language in any voice)

### Use Case Guide

**Choose based on your needs:**

- **Standard TTS (podcasts, audiobooks)**: SpeechT5 or VITS
- **Voice cloning (characters, celebrities)**: XTTS v2
- **Interactive apps (chatbots, assistants)**: VITS (fastest)
- **Creative content (storytelling, entertainment)**: Bark
- **Multilingual content**: XTTS v2 or YourTTS
- **Emotional speech**: Bark

### Example Capabilities

**Bark special syntax:**
```python
# Laughter
"Hello [laughs] this is amazing!"

# Music notation
"And then... [music plays]"

# Different speakers in one generation
"WOMAN: Hello there. MAN: Nice to meet you!"
```

**XTTS v2 voice cloning:**
```python
# Clone voice from reference audio
speaker_wav = "path/to/voice_sample.wav"  # 6+ seconds
tts.tts_to_file("Hello world", speaker_wav=speaker_wav)
```

**💡 Tip**: For production applications requiring custom voices, XTTS v2 is the gold standard. For creative/expressive content, Bark is unmatched. For speed and simplicity, stick with VITS or SpeechT5!

## Exercises

1. **Long Text**: Generate speech for a paragraph (100+ words)
2. **Voice Comparison**: Try all available speaker embeddings and compare
3. **Custom Text**: Generate speech for your own text
4. **Punctuation Effect**: Test how punctuation affects speech prosody
5. **Multiple Languages**: Does the model handle non-English text?

In [ ]:
# Your code here for exercises


## Key Takeaways

✅ **SpeechT5** generates natural-sounding speech

✅ **Speaker embeddings** control voice characteristics

✅ **Vocoder** (HiFiGAN) converts features to waveform

✅ Generated audio is saved as WAV files at 16kHz

✅ Works well for English text

## Next Steps

- Try **Notebook 09**: Multimodal Image-to-Text
- Explore other TTS models on [HuggingFace Hub](https://huggingface.co/models?pipeline_tag=text-to-speech)
- Learn about voice cloning and custom speaker embeddings

## Resources

- [SpeechT5 Paper](https://arxiv.org/abs/2110.07205)
- [TTS Task Guide](https://huggingface.co/docs/transformers/tasks/text-to-speech)
- [SpeechT5 Model Card](https://huggingface.co/microsoft/speecht5_tts)